In [12]:
import re
import csv
from docx import Document

def read_docx(file_path):
    doc = Document(file_path)
    paragraphs = [para.text.strip() for para in doc.paragraphs if para.text.strip() != '']
    return paragraphs

def split_into_sentences(paragraph):
    # Разбиваем абзац на предложения, учитывая точки.
    sentences = re.split(r'(?<!\.\.\.)\.\s+', paragraph)
    # Добавляем точки обратно к предложениям.
    sentences = [s + '.' for s in sentences if s]
    return sentences

def process_paragraph(paragraph):
    sentences = split_into_sentences(paragraph)

    if sentences and re.match(r'^\(.*\)\.$', sentences[-1]):
        sentences.pop()  # Удаляем последнее предложение

    num_sentences = len(sentences)

    if num_sentences < 8:
        return None, {'неправильная строка': paragraph}
    
    if num_sentences == 8:
        return {'предложение_1': sentences[0], 'предложение_2': sentences[1], 'предложение_3': sentences[2],
                'предложение_4': sentences[3], 'предложение_5': sentences[4], 'предложение_6': sentences[5],
                'предложение_7': sentences[6], 'предложение_8': sentences[7]}, None
    
    # Если предложений больше 8
    return {
        'предложение_1': sentences[0],
        'предложение_2': sentences[-7],
        'предложение_3': sentences[-6],
        'предложение_4': sentences[-5],
        'предложение_5': sentences[-4],
        'предложение_6': sentences[-3],
        'предложение_7': sentences[-2],
        'предложение_8': sentences[-1],
        'дополнительные_предложения': ' '.join(sentences[1:-7])
    }, None

def write_csv(file_path, fieldnames, rows):
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)

def main(docx_file, correct_csv, incorrect_csv):
    paragraphs = read_docx(docx_file)
    correct_rows = []
    incorrect_rows = []

    for paragraph in paragraphs:
        correct, incorrect = process_paragraph(paragraph)
        if correct:
            correct_rows.append(correct)
        if incorrect:
            incorrect_rows.append(incorrect)

    correct_fieldnames = [
        'предложение_1', 'предложение_2', 'предложение_3', 'предложение_4', 
        'предложение_5', 'предложение_6', 'предложение_7', 'предложение_8',
        'дополнительные_предложения'
    ]
    
    incorrect_fieldnames = ['неправильная строка']

    write_csv(correct_csv, correct_fieldnames, correct_rows)
    write_csv(incorrect_csv, incorrect_fieldnames, incorrect_rows)

docx_file = 'словарь Венстана_CharisSIL.docx'
correct_csv = 'correct.csv'
incorrect_csv = 'incorrect.csv'

main(docx_file, correct_csv, incorrect_csv)